In [498]:
import warnings
warnings.filterwarnings('ignore')
import copy
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
# Requests sends and recieves HTTP requests.
import pickle
import requests
from collections import defaultdict
# Beautiful Soup parses HTML documents in python.
from bs4 import BeautifulSoup
import time
import random
import re
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
import random
from sklearn.ensemble import RandomForestClassifier
import unicodedata
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.utils import shuffle
from sklearn.ensemble import GradientBoostingRegressor

from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [4]:
data = pd.read_csv('data/zip_code_database.csv')

In [84]:
WA_zips = list(data[data['state'] == 'WA']['zip'].values)
WA_zips

[98001,
 98002,
 98003,
 98004,
 98005,
 98006,
 98007,
 98008,
 98009,
 98010,
 98011,
 98012,
 98013,
 98014,
 98015,
 98019,
 98020,
 98021,
 98022,
 98023,
 98024,
 98025,
 98026,
 98027,
 98028,
 98029,
 98030,
 98031,
 98032,
 98033,
 98034,
 98035,
 98036,
 98037,
 98038,
 98039,
 98040,
 98041,
 98042,
 98043,
 98045,
 98046,
 98047,
 98050,
 98051,
 98052,
 98053,
 98054,
 98055,
 98056,
 98057,
 98058,
 98059,
 98061,
 98062,
 98063,
 98064,
 98065,
 98068,
 98070,
 98071,
 98072,
 98073,
 98074,
 98075,
 98077,
 98082,
 98083,
 98087,
 98089,
 98092,
 98093,
 98101,
 98102,
 98103,
 98104,
 98105,
 98106,
 98107,
 98108,
 98109,
 98110,
 98111,
 98112,
 98113,
 98114,
 98115,
 98116,
 98117,
 98118,
 98119,
 98121,
 98122,
 98124,
 98125,
 98126,
 98127,
 98129,
 98131,
 98132,
 98133,
 98134,
 98136,
 98138,
 98139,
 98141,
 98144,
 98145,
 98146,
 98148,
 98151,
 98154,
 98155,
 98158,
 98160,
 98161,
 98164,
 98165,
 98166,
 98168,
 98170,
 98171,
 98174,
 98175,
 98177,


In [6]:
WA_zips_strings=[]
for zipcode in WA_zips:
    WA_zips_strings.append(str(zipcode))

In [7]:
print(len(WA_zips_strings))
print(len(WA_zips))

733
733


In [232]:
driver = webdriver.Chrome('/Users/ericnaon/Downloads/chromedriver')
#subway_dict = defaultdict(list)
for element in WA_zips[55:]:
    driver.get("https://www.subway.com/en-US/FindAStore?zip=" + str(element))
    iterations = driver.find_elements_by_class_name('locatorResultsPaging')
    try:
        num_iterations = int(iterations[0].text[-1])
    except:
        continue
    for num in range(num_iterations):
        address = driver.find_elements_by_class_name('storeMainAddress')
        address2 = driver.find_elements_by_class_name('locatorAddress2')
        city_and_state = driver.find_elements_by_class_name('locatorAddressCityState')
        for i in range(len(address)):
            adrs = address[i].text.replace('.','')
            city_state_list = city_and_state[i].text.replace(',','').split()
            sub_zip_code = city_state_list[-2][0:6]
            sub_city = city_state_list[0]
            if address2[i].text == '':
                total_address = adrs + ' ' + sub_city + ' ' + sub_zip_code
            else:
                total_address = adrs + ' ' + address2[i].text.replace('-','').replace('#','') + ' ' + sub_city + ' ' + sub_zip_code
            if total_address in subway_dict[sub_zip_code]:
                continue
            else:
                subway_dict[sub_zip_code].append(total_address)
        driver.execute_script("window.scrollTo(0, 500)") 
        time.sleep(4)
        next_page = driver.find_elements_by_class_name('sprite-next-page-arrow')
        if next_page[0].is_displayed():
            next_page[0].click()
        time.sleep(4)
        driver.execute_script("window.scrollTo(0, 50)")
        time.sleep(1)
    

In [233]:
driver.quit()

In [234]:
count = 0
for element in subway_dict.values():
    count += len(element)
print(count)

873


In [236]:
subway_dict

defaultdict(list,
            {'98001': ['1447 W Valley Hwy N Auburn 98001',
              '762 Supermall Way SW Auburn 98001',
              '1101 Outlet Collection Way Suite 1201 Auburn 98001'],
             '98003': ['29017 Military Road South Federal 98003',
              '1928 S Commons C6 Federal 98003',
              '32225 Pacific Hwy South Suite 201 Federal 98003',
              '34520 16th Avenue So Federal 98003',
              '32925 First Ave South Suite M Federal 98003',
              '34815 Pacific Hwy S Suite 200 Federal 98003'],
             '98002': ['4798 Auburn Way N Suite 102 Auburn 98002',
              '1702 Auburn Way North Suite E Auburn 98002',
              '620 Auburn Way S Suite B Auburn 98002',
              '3840 A St SE Suite 101 Auburn 98002'],
             '98032': ['27020 Pacific Hwy S Suite A103 D Kent 98032',
              '1529 W Meeker St 101 Kent 98032',
              '124 North Central 105 Kent 98032',
              '23235 Pacific Coast Hwy Sout

In [237]:
subway_dict_copy = subway_dict.copy()

In [238]:
subway_dict_copy = dict(subway_dict_copy)

In [258]:
subway_locations_dict = subway_dict_copy.copy()
for key, value in subway_dict_copy.items():
    if len(key) > 5:
        subway_locations_dict.pop(key, None)
        subway_locations_dict[key[0:5]] = value

In [259]:
subway_locations_dict

{'98003': ['29017 Military Road South Federal 98003',
  '1928 S Commons C6 Federal 98003',
  '32225 Pacific Hwy South Suite 201 Federal 98003',
  '34520 16th Avenue So Federal 98003',
  '32925 First Ave South Suite M Federal 98003',
  '34815 Pacific Hwy S Suite 200 Federal 98003'],
 '98002': ['4798 Auburn Way N Suite 102 Auburn 98002',
  '1702 Auburn Way North Suite E Auburn 98002',
  '620 Auburn Way S Suite B Auburn 98002',
  '3840 A St SE Suite 101 Auburn 98002'],
 '98032': ['27020 Pacific Hwy S Suite A103 D Kent 98032',
  '1529 W Meeker St 101 Kent 98032',
  '124 North Central 105 Kent 98032',
  '23235 Pacific Coast Hwy South Kent 98032',
  '19918 68th Ave S Kent 98032'],
 '98030': ['25625 104th Avenue SE Street Kent 98030',
  '12925 SE Kent Kangley Rd Ste 104 Kent 98030'],
 '98023': ['1646 Dash Point Rd Suite C, Bldg E Federal 98023',
  '2148 SW 336th St Federal 98023'],
 '98092': ['3312 Auburn Way South Suite M Auburn 98092-',
  '1402 Lake Tapps Pkwy E Suite 108 Bldg F Auburn 9809

In [265]:
subway_locations_dict2 = subway_locations_dict.copy()
for key in subway_locations_dict.keys():
    if key not in WA_zips_strings:
        print(key)
        subway_locations_dict2.pop(key, None)

8M9
8K1
2K8
8H6
3N3
1C4
2B9
5A1
4K9
1V5
6K9
4E6
0C5
3N5
2Z7
0M6
1A7
5M4
6R3
0A3
5Y1
5N4
1J8
2C4
5J2
5P5
3K4
5E4
2L1
6E3
3E9
3L5
2E8
2H7
2W4
2A6
2G6
4W6
3X1
2X9
1K8
4H6
9N6
4N3
3Z3
0C8
4G7
2J4
3N7
1A5
3V2
2A2
1C6
1N3
7E9
5E7
9A5
1P3
1P2
8E7
1M9
3B9
1T0
2G2
4Z3
2P2
0G4
2N4
2T2
0Y3
2K5
3G3
2V8
5B5
0R6
2A9
6P5
3K8
3C7
5R8
1X0
3Z9
7V2
2P6
1R1
5A7
5G5
3M4
2P8
1H8
3R5
2P0
2X3
1X1
4C6
5E3
6T1
1H3
2W0
2S1
1A0
2A4
0B2
1W4
1E7
1K2
1W6
2Z3
3X9
1V2
2L7
6G6
0M5
2C7
1P6
5V2
1J4
1P8
2W8
1Y1
4C1
5X2
1C7
1L2
1L0
1W0
0A5
2G0
3R2
6C6
1K0
1B3
7A4
5S5
1P1
5E2
4W3
97031
1H1
97103
97146
97138
97016
97048
97064
97051
97056
97060
97220
97230
97217
97030
97218
97058
97067
97055
97233
97236
97211
97203
97213
97216
97106
97229
97124
97113
97123
97818
97023
97141
97741
97015
97882
97838
97875
97210
97232
97209
97225
97204
97205
97214
97201
97266
1V0
1N2
1H0
9B8
6W6
5G8
5L1
1Z0
1X5
1Y9
2Z1
3G8
4C9
5V6
6C7
6H2
8E6
6M4
7C6
7K2
1X6
5C3
1V7
3C3
1S1
1P5
1Y0
4A3
4W4
1B8
9R1
2W9
1G8
9H6
4Y4
5Z3
83854
83815
83835
83814
8386

In [272]:
count = 0
for element in subway_locations_dict2.values():
    count += len(element)
print('Number of Subways in WA:',count)

Number of Subways in WA: 551


In [273]:
#with open('subway_dict_WA.pickle', 'wb') as handle:
    #pickle.dump(subway_locations_dict2, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [833]:
with open('data/subway_dict_WA.pickle', 'rb') as handle:
    subway_dict_WA = pickle.load(handle)

In [834]:
business_data = pd.read_csv('data/zbp16detail.txt')

In [835]:
def rename_cols(columns,string):
    new_col = []
    for col in columns:
        new_col.append(string+col)
    return new_col

In [836]:
df = business_data[(business_data['zip'] >= 98001) & (business_data['zip'] <= 99403)].groupby('zip').sum()
df.columns = rename_cols(df.columns,'num_businesses_')

In [837]:
df

,num_businesses_est,num_businesses_n1_4,num_businesses_n5_9,num_businesses_n10_19,num_businesses_n20_49,num_businesses_n50_99,num_businesses_n100_249,num_businesses_n250_499,num_businesses_n500_999,num_businesses_n1000
zip,,,,,,,,,,
98001,6828,3162,1230,1050,834,288,186,36,24,18
98002,5858,3074,1116,798,630,108,90,36,0,6
98003,9576,4992,1968,1242,858,336,132,36,12,0
98004,17966,9236,3162,2256,1764,792,498,156,66,36
98005,10156,5266,1938,1434,864,372,204,60,12,6
...,...,...,...,...,...,...,...,...,...,...
99363,102,30,12,24,12,6,6,6,0,6
99371,18,12,6,0,0,0,0,0,0,0
99401,6,0,6,0,0,0,0,0,0,0


In [838]:
for zipcode in WA_zips:
    if zipcode not in df.index:
        df.loc[zipcode,:] = -1

In [839]:
subway_count = dict()
for key, value in subway_dict_WA.items():
    subway_count[int(key)] = len(value)

In [840]:
df['subway_count'] = pd.Series(subway_count)

In [841]:
df = df.fillna(0)

In [842]:
df

,num_businesses_est,num_businesses_n1_4,num_businesses_n5_9,num_businesses_n10_19,num_businesses_n20_49,num_businesses_n50_99,num_businesses_n100_249,num_businesses_n250_499,num_businesses_n500_999,num_businesses_n1000,subway_count
zip,,,,,,,,,,,
98001,6828.0,3162.0,1230.0,1050.0,834.0,288.0,186.0,36.0,24.0,18.0,0.0
98002,5858.0,3074.0,1116.0,798.0,630.0,108.0,90.0,36.0,0.0,6.0,4.0
98003,9576.0,4992.0,1968.0,1242.0,858.0,336.0,132.0,36.0,12.0,0.0,6.0
98004,17966.0,9236.0,3162.0,2256.0,1764.0,792.0,498.0,156.0,66.0,36.0,5.0
98005,10156.0,5266.0,1938.0,1434.0,864.0,372.0,204.0,60.0,12.0,6.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...
98667,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0
98190,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0
99144,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0


In [843]:
num_limited_service_rest_df = business_data[(business_data['naics'] == '722513') & (business_data['zip'] >= 98001) & (business_data['zip'] <= 99403)].groupby('zip').sum()
num_limited_service_rest_df.columns = rename_cols(num_limited_service_rest_df.columns,'num_limited_ser_rest_')
df = df.join(num_limited_service_rest_df,how='left')

In [844]:
df

,num_businesses_est,num_businesses_n1_4,num_businesses_n5_9,num_businesses_n10_19,num_businesses_n20_49,num_businesses_n50_99,num_businesses_n100_249,num_businesses_n250_499,num_businesses_n500_999,num_businesses_n1000,...,num_limited_ser_rest_est,num_limited_ser_rest_n1_4,num_limited_ser_rest_n5_9,num_limited_ser_rest_n10_19,num_limited_ser_rest_n20_49,num_limited_ser_rest_n50_99,num_limited_ser_rest_n100_249,num_limited_ser_rest_n250_499,num_limited_ser_rest_n500_999,num_limited_ser_rest_n1000
zip,,,,,,,,,,,,,,,,,,,,,
98001,6828.0,3162.0,1230.0,1050.0,834.0,288.0,186.0,36.0,24.0,18.0,...,24.0,8.0,3.0,8.0,4.0,1.0,0.0,0.0,0.0,0.0
98002,5858.0,3074.0,1116.0,798.0,630.0,108.0,90.0,36.0,0.0,6.0,...,40.0,11.0,5.0,6.0,15.0,3.0,0.0,0.0,0.0,0.0
98003,9576.0,4992.0,1968.0,1242.0,858.0,336.0,132.0,36.0,12.0,0.0,...,62.0,13.0,10.0,17.0,19.0,1.0,2.0,0.0,0.0,0.0
98004,17966.0,9236.0,3162.0,2256.0,1764.0,792.0,498.0,156.0,66.0,36.0,...,59.0,21.0,7.0,15.0,14.0,2.0,0.0,0.0,0.0,0.0
98005,10156.0,5266.0,1938.0,1434.0,864.0,372.0,204.0,60.0,12.0,6.0,...,23.0,12.0,8.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98667,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98190,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99144,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [845]:
num_restaurants_df = business_data[(business_data['naics'] == '72251/') & (business_data['zip'] >= 98001) & (business_data['zip'] <= 99403)].groupby('zip').sum()
num_restaurants_df.columns = rename_cols(num_restaurants_df.columns,'num_rest_')
df = df.join(num_restaurants_df,how='left')

In [846]:
df

,num_businesses_est,num_businesses_n1_4,num_businesses_n5_9,num_businesses_n10_19,num_businesses_n20_49,num_businesses_n50_99,num_businesses_n100_249,num_businesses_n250_499,num_businesses_n500_999,num_businesses_n1000,...,num_rest_est,num_rest_n1_4,num_rest_n5_9,num_rest_n10_19,num_rest_n20_49,num_rest_n50_99,num_rest_n100_249,num_rest_n250_499,num_rest_n500_999,num_rest_n1000
zip,,,,,,,,,,,,,,,,,,,,,
98001,6828.0,3162.0,1230.0,1050.0,834.0,288.0,186.0,36.0,24.0,18.0,...,50.0,16.0,7.0,15.0,10.0,2.0,0.0,0.0,0.0,0.0
98002,5858.0,3074.0,1116.0,798.0,630.0,108.0,90.0,36.0,0.0,6.0,...,74.0,23.0,9.0,13.0,26.0,3.0,0.0,0.0,0.0,0.0
98003,9576.0,4992.0,1968.0,1242.0,858.0,336.0,132.0,36.0,12.0,0.0,...,157.0,51.0,30.0,29.0,34.0,11.0,2.0,0.0,0.0,0.0
98004,17966.0,9236.0,3162.0,2256.0,1764.0,792.0,498.0,156.0,66.0,36.0,...,175.0,39.0,21.0,42.0,44.0,18.0,11.0,0.0,0.0,0.0
98005,10156.0,5266.0,1938.0,1434.0,864.0,372.0,204.0,60.0,12.0,6.0,...,54.0,22.0,14.0,11.0,5.0,2.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98667,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98190,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99144,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [847]:
pop_data = pd.read_csv('/Users/ericnaon/Downloads/ACS_17_5YR_S0101/ACS_17_5YR_S0101_with_ann.csv')

In [848]:
cols=list(pop_data.iloc[0,:].values)

In [849]:
pop_data.iloc[0,:].values

array(['Id', 'Id2', 'Geography', 'Total; Estimate; Total population',
       'Total; Margin of Error; Total population',
       'Percent; Estimate; Total population',
       'Percent; Margin of Error; Total population',
       'Male; Estimate; Total population',
       'Male; Margin of Error; Total population',
       'Percent Male; Estimate; Total population',
       'Percent Male; Margin of Error; Total population',
       'Female; Estimate; Total population',
       'Female; Margin of Error; Total population',
       'Percent Female; Estimate; Total population',
       'Percent Female; Margin of Error; Total population',
       'Total; Estimate; AGE - Under 5 years',
       'Total; Margin of Error; AGE - Under 5 years',
       'Percent; Estimate; AGE - Under 5 years',
       'Percent; Margin of Error; AGE - Under 5 years',
       'Male; Estimate; AGE - Under 5 years',
       'Male; Margin of Error; AGE - Under 5 years',
       'Percent Male; Estimate; AGE - Under 5 years',
       'P

In [850]:
for i in list(np.arange(0,85,5)):
    if i == 0:
        string = 'Total; Estimate; AGE - ' + 'Under 5 years'
        index = cols.index(string)
    elif i == 84:
        string = 'Total; Estimate; AGE - ' + str(i) +' years and over'
        index = cols.index(string)
    else:
        string = 'Total; Estimate; AGE - ' + str(i) +' to ' + str(i+4) + ' years'
        index = cols.index(string)
    series = pd.Series(pop_data.iloc[:,index].values,index = pop_data['GEO.id2']).iloc[1:]
    series.index = series.index.astype(int)
    df[string] = series

In [851]:
df

,num_businesses_est,num_businesses_n1_4,num_businesses_n5_9,num_businesses_n10_19,num_businesses_n20_49,num_businesses_n50_99,num_businesses_n100_249,num_businesses_n250_499,num_businesses_n500_999,num_businesses_n1000,...,Total; Estimate; AGE - 35 to 39 years,Total; Estimate; AGE - 40 to 44 years,Total; Estimate; AGE - 45 to 49 years,Total; Estimate; AGE - 50 to 54 years,Total; Estimate; AGE - 55 to 59 years,Total; Estimate; AGE - 60 to 64 years,Total; Estimate; AGE - 65 to 69 years,Total; Estimate; AGE - 70 to 74 years,Total; Estimate; AGE - 75 to 79 years,Total; Estimate; AGE - 80 to 84 years
zip,,,,,,,,,,,,,,,,,,,,,
98001,6828.0,3162.0,1230.0,1050.0,834.0,288.0,186.0,36.0,24.0,18.0,...,1997,2214,2323,2541,2645,2206,1503,906,728,332
98002,5858.0,3074.0,1116.0,798.0,630.0,108.0,90.0,36.0,0.0,6.0,...,2625,2401,1911,2297,2552,1848,1201,981,836,517
98003,9576.0,4992.0,1968.0,1242.0,858.0,336.0,132.0,36.0,12.0,0.0,...,2534,2738,2934,3448,3114,2728,2044,1561,1274,849
98004,17966.0,9236.0,3162.0,2256.0,1764.0,792.0,498.0,156.0,66.0,36.0,...,2674,2106,2644,2384,2055,1797,1501,1013,937,741
98005,10156.0,5266.0,1938.0,1434.0,864.0,372.0,204.0,60.0,12.0,6.0,...,1684,1509,1436,1039,1323,679,699,805,531,341
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98667,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98190,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99144,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0,4,4,0,0,7,0,0,0,0


In [852]:
def add_more_cols_to_df(dataframe,target_df,target_col,name_zip_col,name_new_col):
    series = pd.Series(target_df[target_col].values,index = target_df[name_zip_col]).iloc[1:]
    series.index = series.index.astype(int)
    dataframe[name_new_col] = series
    return dataframe

In [853]:
df = add_more_cols_to_df(df,pop_data,'HC01_EST_VC01','GEO.id2','population')

In [854]:
income_data = pd.read_csv('/Users/ericnaon/Downloads/ACS_17_5YR_S1903/ACS_17_5YR_S1903_with_ann.csv')

In [855]:
df = add_more_cols_to_df(df,income_data,'HC03_EST_VC02','GEO.id2','median_income')
df = add_more_cols_to_df(df,income_data,'HC01_EST_VC02','GEO.id2','num_households')

In [856]:
lang_data = pd.read_csv('/Users/ericnaon/Downloads/ACS_17_5YR_S1601/ACS_17_5YR_S1601_with_ann.csv')

In [857]:
lang_data.columns = lang_data.iloc[0,:].values
lang_data

,Id,Id2,Geography,Total; Estimate; Population 5 years and over,Total; Margin of Error; Population 5 years and over,Percent; Estimate; Population 5 years and over,Percent; Margin of Error; Population 5 years and over,"Percent of specified language speakers - Speak English only or speak English ""very well""; Estimate; Population 5 years and over","Percent of specified language speakers - Speak English only or speak English ""very well""; Margin of Error; Population 5 years and over","Percent of specified language speakers - Percent speak English only or speak English ""very well""; Estimate; Population 5 years and over",...,Percent; Estimate; CITIZENS 18 YEARS AND OVER - All citizens 18 years old and over - Speak a language other than English - Other languages,Percent; Margin of Error; CITIZENS 18 YEARS AND OVER - All citizens 18 years old and over - Speak a language other than English - Other languages,"Percent of specified language speakers - Speak English only or speak English ""very well""; Estimate; CITIZENS 18 YEARS AND OVER - All citizens 18 years old and over - Speak a language other than English - Other languages","Percent of specified language speakers - Speak English only or speak English ""very well""; Margin of Error; CITIZENS 18 YEARS AND OVER - All citizens 18 years old and over - Speak a language other than English - Other languages","Percent of specified language speakers - Percent speak English only or speak English ""very well""; Estimate; CITIZENS 18 YEARS AND OVER - All citizens 18 years old and over - Speak a language other than English - Other languages","Percent of specified language speakers - Percent speak English only or speak English ""very well""; Margin of Error; CITIZENS 18 YEARS AND OVER - All citizens 18 years old and over - Speak a language other than English - Other languages","Percent of specified language speakers - Speak English less than ""very well""; Estimate; CITIZENS 18 YEARS AND OVER - All citizens 18 years old and over - Speak a language other than English - Other languages","Percent of specified language speakers - Speak English less than ""very well""; Margin of Error; CITIZENS 18 YEARS AND OVER - All citizens 18 years old and over - Speak a language other than English - Other languages","Percent of specified language speakers - Percent speak English less than ""very well""; Estimate; CITIZENS 18 YEARS AND OVER - All citizens 18 years old and over - Speak a language other than English - Other languages","Percent of specified language speakers - Percent speak English less than ""very well""; Margin of Error; CITIZENS 18 YEARS AND OVER - All citizens 18 years old and over - Speak a language other than English - Other languages"
0,Id,Id2,Geography,Total; Estimate; Population 5 years and over,Total; Margin of Error; Population 5 years and...,Percent; Estimate; Population 5 years and over,Percent; Margin of Error; Population 5 years a...,Percent of specified language speakers - Spea...,Percent of specified language speakers - Spea...,Percent of specified language speakers - Perc...,...,Percent; Estimate; CITIZENS 18 YEARS AND OVER ...,Percent; Margin of Error; CITIZENS 18 YEARS AN...,Percent of specified language speakers - Spea...,Percent of specified language speakers - Spea...,Percent of specified language speakers - Perc...,Percent of specified language speakers - Perc...,Percent of specified language speakers - Spea...,Percent of specified language speakers - Spea...,Percent of specified language speakers - Perc...,Percent of specified language speakers - Perc...
1,8600000US83501,83501,ZCTA5 83501,32821,337,(X),(X),32511,372,99.1,...,1.7,0.7,397,174,83.4,10.0,79,46,16.6,10.0
2,8600000US83822,83822,ZCTA5 83822,1769,264,(X),(X),1755,265,99.2,...,0.3,0.6,5,9,100.0,100.0,0,12,0.0,100.0
3,8600000US83824,83824,ZCTA5 83824,285,64,(X),(X),285,64,100.0,...,5.2,4.2,12,10,100.0,85.6,0,12,0.0,85.6
4,8600000US83832,83832,ZCTA5 83832,1425,122,(X),(X),1401,122,98.3,...,1.2,0.9,6,6,46.2,30.4,7,7,5

In [858]:
lang_data.iloc[:,9].name

'Percent of specified language speakers  - Percent speak English only or speak English "very well"; Estimate; Population 5 years and over'

In [859]:
df = add_more_cols_to_df(df,lang_data,'Percent; Estimate; SPEAK A LANGUAGE OTHER THAN ENGLISH - Spanish','Id2','spanish_speaking')
df = add_more_cols_to_df(df,lang_data,'Percent; Estimate; SPEAK A LANGUAGE OTHER THAN ENGLISH - Other Indo-European languages','Id2','indo_euro_speaking')
df = add_more_cols_to_df(df,lang_data,'Percent; Estimate; SPEAK A LANGUAGE OTHER THAN ENGLISH - Asian and Pacific Island languages','Id2','asian_pacific_speaking')
df = add_more_cols_to_df(df,lang_data,'Percent; Estimate; SPEAK A LANGUAGE OTHER THAN ENGLISH - Other languages','Id2','other_languages')
df = add_more_cols_to_df(df,lang_data,'Percent of specified language speakers  - Percent speak English only or speak English "very well"; Estimate; Population 5 years and over','Id2','percent_english_speaking')
df

,num_businesses_est,num_businesses_n1_4,num_businesses_n5_9,num_businesses_n10_19,num_businesses_n20_49,num_businesses_n50_99,num_businesses_n100_249,num_businesses_n250_499,num_businesses_n500_999,num_businesses_n1000,...,Total; Estimate; AGE - 75 to 79 years,Total; Estimate; AGE - 80 to 84 years,population,median_income,num_households,spanish_speaking,indo_euro_speaking,asian_pacific_speaking,other_languages,percent_english_speaking
zip,,,,,,,,,,,,,,,,,,,,,
98001,6828.0,3162.0,1230.0,1050.0,834.0,288.0,186.0,36.0,24.0,18.0,...,728,332,32789,77083,11097,8.5,11.1,8.5,0.8,88.5
98002,5858.0,3074.0,1116.0,798.0,630.0,108.0,90.0,36.0,0.0,6.0,...,836,517,34255,51086,13111,17.7,4.8,7.0,0.6,87.9
98003,9576.0,4992.0,1968.0,1242.0,858.0,336.0,132.0,36.0,12.0,0.0,...,1274,849,48065,53023,18279,15.0,6.6,12.5,3.5,82.1
98004,17966.0,9236.0,3162.0,2256.0,1764.0,792.0,498.0,156.0,66.0,36.0,...,937,741,34283,117321,16006,3.3,12.3,20.7,1.1,88.1
98005,10156.0,5266.0,1938.0,1434.0,864.0,372.0,204.0,60.0,12.0,6.0,...,531,341,18735,101134,7761,5.3,10.5,22.2,2.3,83.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98667,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98190,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99144,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0,0,23,-,8,0.0,0.0,0.0,0.0,100.0


In [860]:
for col in df.columns[-5:]:
    df[col] = df[col].apply(lambda x: '0' if x == '-' else x)

In [861]:
#education_data = pd.read_csv('/Users/ericnaon/Downloads/ACS_17_5YR_B07009/ACS_17_5YR_B07009_with_ann.csv')

In [862]:
#below makes model worse
#df = add_more_cols_to_df(df,education_data,'HD01_VD03','GEO.id2','less_than_hs')
#df = add_more_cols_to_df(df,education_data,'HD01_VD04','GEO.id2','hs_graduate')
#df = add_more_cols_to_df(df,education_data,'HD01_VD05','GEO.id2','some_college')
#df = add_more_cols_to_df(df,education_data,'HD01_VD06','GEO.id2','bachelors')
#df = add_more_cols_to_df(df,education_data,'HD01_VD07','GEO.id2','graduate_or_pro')
#df

In [863]:
model = GradientBoostingRegressor(learning_rate=.01, criterion='mae',max_depth=5)

In [864]:
df.dropna(inplace=True)

In [865]:
med_ints = list()
for element in list(df['median_income'].values):
    if '-' in element:
        med_ints.append(0)
    elif '+' in element:
        med_ints.append(int(element.replace('+','').replace(',','')))
    else:
        med_ints.append(int(element))

In [866]:
df['median_income'] = med_ints

In [867]:
for col in df.columns:
    df[col] = df[col].astype(float)

In [868]:
y = df['subway_count']
df.drop(columns='subway_count',inplace=True)

In [869]:
#y = y.to_numpy()
#df = df.to_numpy()

In [870]:
train_data, test_data, y_train, y_test = train_test_split(df, y)

In [871]:
model.fit(train_data,y_train)
model.predict(test_data)

array([2.37513727, 0.59835056, 1.81358204, 1.8098474 , 1.17961438,
       0.54564672, 2.95429157, 0.54564672, 1.9916296 , 1.89982064,
       1.81358204, 1.17961438, 0.54564672, 1.17961438, 2.22812362,
       2.8803389 , 0.54564672, 2.90524268, 0.69721473, 0.76983846,
       1.17961438, 1.17961438, 2.25468769, 1.17961438, 1.70036299,
       0.54564672, 0.93280177, 1.17961438, 1.81358204, 1.17961438,
       1.17961438, 1.17961438, 1.19334903, 0.54564672, 1.17961438,
       1.17961438, 0.54564672, 1.3246811 , 3.02293253, 2.21866454,
       1.17961438, 1.18334903, 1.94773705, 1.17961438, 1.79436304,
       0.76194196, 1.79120037, 1.24139995, 0.56693422, 0.54564672,
       1.17961438, 0.54564672, 0.68565596, 0.54564672, 0.54564672,
       1.17961438, 1.21526177, 2.06116953, 1.34945958, 2.29286788,
       1.24743444, 0.54564672, 1.17961438, 1.81358204, 0.54564672,
       1.17961438, 2.2957472 , 2.02014621, 2.16964467, 1.8098474 ,
       1.18334903, 1.40312126, 2.02046432, 0.70571755, 0.54564

pandas.core.frame.DataFrame

In [876]:
yhat = model.predict(test_data)
print(max((yhat-y_test)))
np.argmax(yhat-y_test)

1.905242683503365


99362

In [787]:
kf = KFold(n_splits=7)

In [788]:
model_scores = []
for train, test in kf.split(train_data):
    X_train, X_test, y_train_kf, y_test_kf = train_data[train], train_data[test], y_train[train], y_train[test]
    model.fit(X_train,y_train_kf)
    mae = mean_absolute_error(y_test_kf, model.predict(X_test))
    model_scores.append(mae)

In [789]:
np.average(model_scores)

0.7442200353374029

In [790]:
model.fit(X_train,y_train)

ValueError: Found input variables with inconsistent numbers of samples: [231, 269, 231]